<a href="https://colab.research.google.com/github/shaunwang1350/MyersBriggsMLProject/blob/master/Notebooks/Data_Preprocessing_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up environment

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.7'
spark_version = 'spark-2.4.7'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [1

In [2]:
# download postgres driver
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-10-07 18:21:30--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.03MB/s    in 0.8s    

2020-10-07 18:21:32 (1.03 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Import the data from S3 buckets

In [13]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
import pandas as pd

url ="https://myersbriggsml.s3.us-east-2.amazonaws.com/resources/mbti_raw.csv"
spark.sparkContext.addFile(url)
raw_df = spark.read.csv(SparkFiles.get("mbti_raw.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
raw_df.show(10)

+----+--------------------+
|type|               posts|
+----+--------------------+
|INFJ|'http://www.youtu...|
|ENTP|'I'm finding the ...|
|INTP|'Good one  _____ ...|
|INTJ|'Dear INTP,   I e...|
|ENTJ|'You're fired.|||...|
|INTJ|'18/37 @.@|||Scie...|
|INFJ|'No, I can't draw...|
|INTJ|'I tend to build ...|
|INFJ|I'm not sure, tha...|
|INTP|'https://www.yout...|
+----+--------------------+
only showing top 10 rows



# Data Preprocessing

In [31]:
myersBriggs_df = raw_df
myersBriggs_df.show(10)

+----+--------------------+
|type|               posts|
+----+--------------------+
|INFJ|'http://www.youtu...|
|ENTP|'I'm finding the ...|
|INTP|'Good one  _____ ...|
|INTJ|'Dear INTP,   I e...|
|ENTJ|'You're fired.|||...|
|INTJ|'18/37 @.@|||Scie...|
|INFJ|'No, I can't draw...|
|INTJ|'I tend to build ...|
|INFJ|I'm not sure, tha...|
|INTP|'https://www.yout...|
+----+--------------------+
only showing top 10 rows



In [ ]:
# Filter for URLs

In [ ]:
# Filter for Stopwords

# Additional Features

In [35]:
# Import functions
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
from pyspark.sql.functions import length
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes

In [36]:
# Create a length column to be used as a future feature
myersBriggs_df = myersBriggs_df.withColumn('length', length(myersBriggs_df['posts']))
myersBriggs_df.show(10)

+----+--------------------+------+
|type|               posts|length|
+----+--------------------+------+
|INFJ|'http://www.youtu...|  4652|
|ENTP|'I'm finding the ...|  7053|
|INTP|'Good one  _____ ...|  5265|
|INTJ|'Dear INTP,   I e...|  6271|
|ENTJ|'You're fired.|||...|  6111|
|INTJ|'18/37 @.@|||Scie...|  8589|
|INFJ|'No, I can't draw...|  7916|
|INTJ|'I tend to build ...|  6900|
|INFJ|I'm not sure, tha...|  5325|
|INTP|'https://www.yout...|  7573|
+----+--------------------+------+
only showing top 10 rows



In [ ]:
# Create a stopwords column to be used as a future feature

In [ ]:
# Create a punctuation column to be used as a future feature

# RDS Upload



In [ ]:
# # Configure settings for RDS
# mode = "append"
# jdbc_url="jdbc:postgresql://:5432/"
# config = {"user":"sellbyprojectdb", 
#           "password": "Sellproject!@", 
#           "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to active_user table in RDS
# review_id_table_df.write.jdbc(url=jdbc_url, table='review_id', mode=mode, properties=config)
# products_df.write.jdbc(url=jdbc_url, table='products_df', mode=mode, properties=config)
# customers_df.write.jdbc(url=jdbc_url, table='customers_df', mode=mode, properties=config)
# vine_table_df.write.jdbc(url=jdbc_url, table='vine_table_df', mode=mode, properties=config)